<a href="https://colab.research.google.com/github/isabellasims/Google-colabs/blob/master/geopandas_hate_crimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

references:

https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/


How to do dc district lining:
https://rsandstroem.github.io/GeoMapsFoliumDemo.html

GeoJSON for dc districs:

https://github.com/unitedstates/districts/blob/gh-pages/states/DC/sldl/Ward-1.geojson

In [3]:
import pandas as pd
!pip install geopandas
import geopandas as gpd #rename as gpd
import math

In [4]:
import folium # docs: https://python-visualization.github.io/folium/quickstart.html#Getting-Started
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [6]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")

In [126]:
# Load the data
# data from http://crimemap.dc.gov/Download.aspx
crimes = pd.read_csv("/HateCrimes.csv", encoding='latin-1')

# this step needs to be done to the original dataframe
crimes['STREET'] = crimes['Block Location'] + 'WASHINGTON DC'


# rename columns for consistency and easy of access. note inplace=True param will not make a copy, it will replace. Avoids slice error later
crimes.rename(columns={"Date of Offense": "DATE_OFFENSE",'Time of Offense':'TIME_OFFENSE','Date Offense Reported':'DATE_REPORT','Report Year':'REPORT_YEAR','Month':'MONTH','District':"DISTRICT","Block Location":"BLOCK",'Type of Hate Bias':'BIAS','Targeted Group':'GROUP','Top Offense Type':'TOP_OFFENSE_TYPE'},inplace=True)
# for some reason targetet group refused to be renamed

# Grab relevant columns and add we'll make later to use later
columnsTitles = ['STREET','DATE_OFFENSE', 'REPORT_YEAR', 'DISTRICT','BLOCK','LAT','LONG','LOC','BIAS','Targeted Group','TOP_OFFENSE_TYPE']
# create df of selected titles only
crimes = crimes.reindex(columns=columnsTitles)


crimes.head()

,STREET,DATE_OFFENSE,REPORT_YEAR,DISTRICT,BLOCK,LAT,LONG,LOC,BIAS,Targeted Group,TOP_OFFENSE_TYPE
0,1600 B/O 17th St NW WASHINGTON DC,1/8/12,2012,3D,1600 B/O 17th St NW,NaN,NaN,NaN,Sexual Orientation,NaN,Threats
1,3rd St SW & K St SWWASHINGTON DC,1/12/12,2012,1D,3rd St SW & K St SW,NaN,NaN,NaN,Sexual Orientation,NaN,Robbery
2,Park Rd NW & Sherman Ave NWWASHINGTON DC,1/13/12,2012,4D,Park Rd NW & Sherman Ave NW,NaN,NaN,NaN,Race,NaN,Simple Assault
3,1800 B/O 14th St NWWASHINGTON DC,1/14/12,2012,3D,1800 B/O 14th St NW,NaN,NaN,NaN,Sexual Orientation,NaN,Simple Assault
4,18th St NW & Florida Ave NWWASHINGTON DC,1/14/12,2012,3D,18th St NW & Florida Ave NW,NaN,NaN,NaN,Ethnicity/National Origin,NaN,Simple Assault


In [120]:
crimes.head()

,STREET,DATE_OFFENSE,REPORT_YEAR,DISTRICT,BLOCK,LAT,LONG,LOC,BIAS,Targeted Group,TOP_OFFENSE_TYPE
0,1600 B/O 17th St NW WASHINGTON DC,1/8/12,2012,3D,1600 B/O 17th St NW,NaN,NaN,NaN,Sexual Orientation,NaN,Threats
1,3rd St SW & K St SWWASHINGTON DC,1/12/12,2012,1D,3rd St SW & K St SW,NaN,NaN,NaN,Sexual Orientation,NaN,Robbery
2,Park Rd NW & Sherman Ave NWWASHINGTON DC,1/13/12,2012,4D,Park Rd NW & Sherman Ave NW,NaN,NaN,NaN,Race,NaN,Simple Assault
3,1800 B/O 14th St NWWASHINGTON DC,1/14/12,2012,3D,1800 B/O 14th St NW,NaN,NaN,NaN,Sexual Orientation,NaN,Simple Assault
4,18th St NW & Florida Ave NWWASHINGTON DC,1/14/12,2012,3D,18th St NW & Florida Ave NW,NaN,NaN,NaN,Ethnicity/National Origin,NaN,Simple Assault


In [77]:
racialBias = crimes[crimes.BIAS == 'Race'] # create homicide df


# add geocoded location, lat, and long points to df
# def addLoc(x):
#      return geolocator.geocode(x)

# def addLat(loc):
#     return geolocator.geocode(loc).latitude

# def addLong(loc):
#   return geolocator.geocode(loc).longitude

#racialBias['LOC'] = racialBias['STREET'].apply(addLoc)
#racialBias['LAT'] = crimes['LOC'].apply(addLat)
#racialBias['LONG'] = crimes['LOC'].apply(addLong)


In [74]:
racialBias.head()

,STREET,DATE_OFFENSE,REPORT_YEAR,DISTRICT,BLOCK,LAT,LONG,LOC,BIAS,Targeted Group,TOP_OFFENSE_TYPE
2,NaN,1/13/12,2012,4D,Park Rd NW & Sherman Ave NW,NaN,NaN,NaN,Race,NaN,Simple Assault
6,NaN,1/22/12,2012,1D,600 B/O H St NW,NaN,NaN,NaN,Race,NaN,Threats
8,NaN,2/8/12,2012,1D,400 B/O 3rd St SE,NaN,NaN,NaN,Race,NaN,Threats
25,NaN,4/6/12,2012,2D,1300 B/O Pennsylvania Ave NW,NaN,NaN,NaN,Race,NaN,Threats
26,NaN,4/29/12,2012,2D,22nd St NW & P St NW,NaN,NaN,NaN,Race,NaN,Damage/Defacing/Destruction of Property


In [124]:
# Create a map
m_2 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

# # Add points to the map
# for idx, row in homicides.iterrows():
#     Marker([row['LAT'], row['LONG']]).add_to(m_2)

# # Display the map


In [134]:
kanton_map = folium.Map(location=[38.9072,-77.0369],
                   tiles='Mapbox Bright', zoom_start=13)
kanton_map

# note: need param geo_data, not geo_path from tutorial
kanton_map.choropleth(geo_data='/content/Washington_DC_Boundary (1).geojson')
kanton_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [ ]:
m_3 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

nightTimeHomicides = homicides[homicides.SHIFT == 'MIDNIGHT']


# Add points to the map
mc = MarkerCluster()
for idx, row in nightTimeHomicides.iterrows():
    if not math.isnan(row['LONG']) and not math.isnan(row['LAT']):
        mc.add_child(Marker([row['LAT'], row['LONG']]))
m_3.add_child(mc)

# Display the map
m_3

In [ ]:
m_3 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

#ward1 = '{"type":"Feature","properties":{"ocdid":"ocd-division/country:us/district:dc/ward:1","type":"sldu","state":"dc","name":"Ward 1"},"geometry":{"type":"Polygon","coordinates":[[[-77.050779,38.93300999907752],[-77.050503,38.933223999077505],[-77.050424,38.933188999077515],[-77.050354,38.933233999077515],[-77.050244,38.9333039990775],[-77.050137,38.93336599907751],[-77.050021,38.933442999077506],[-77.049933,38.93348899907752],[-77.049902,38.93350699907751],[-77.049871,38.93352299907751],[-77.049839,38.93353699907752],[-77.049776,38.933568999077515],[-77.049744,38.93358699907751],[-77.04971,38.93360299907751],[-77.049645,38.93363699907751],[-77.049614,38.933656999077506],[-77.04955,38.9336889990775],[-77.049517,38.933701999077506],[-77.049483,38.933713999077504],[-77.049448,38.93372799907751],[-77.049411,38.9337379990775],[-77.049374,38.9337449990775],[-77.04934,38.93375299907751],[-77.049306,38.93376299907751],[-77.049242,38.933784999077496],[-77.049188,38.933814999077505],[-77.049122,38.9338709990775],[-77.049066,38.9339349990775],[-77.049044,38.9339569990775],[-77.049,38.9339939990775],[-77.048954,38.93402799907751],[-77.048904,38.934068999077496],[-77.048865,38.93409699907751],[-77.04881,38.9341409990775],[-77.048713,38.9342159990775],[-77.048635,38.934292999077506],[-77.048612,38.93431399907751],[-77.048589,38.934336999077495],[-77.048566,38.934363999077505],[-77.04852,38.934414999077504],[-77.048466,38.93446599907751],[-77.048442,38.93449399907751],[-77.048395,38.9345419990775],[-77.048375,38.9345679990775],[-77.048355,38.93459199907749],[-77.048329,38.9346279990775],[-77.048314,38.9346569990775],[-77.048287,38.934684999077504],[-77.048261,38.93471499907751],[-77.048225,38.9347399990775],[-77.048172,38.9347729990775],[-77.048135,38.93479899907751],[-77.048097,38.9348199990775],[-77.048071,38.9348379990775],[-77.048032,38.93487999907751],[-77.048005,38.93491199907749],[-77.04798,38.9349389990775],[-77.047942,38.93498599907749],[-77.047897,38.935013999077505],[-77.047844,38.935025999077496],[-77.047802,38.93510699907749],[-77.04769,38.9351599990775],[-77.047713,38.9350529990775],[-77.047569,38.9351089990775],[-77.046753,38.935349999077495],[-77.046663,38.93537799907749],[-77.046529,38.935422999077495],[-77.046441,38.93545599907749],[-77.046161,38.935568999077496],[-77.045845,38.935700999077504],[-77.04559,38.93580899907749],[-77.045533,38.935834999077485],[-77.045503,38.9358459990775],[-77.045474,38.93585799907749],[-77.045414,38.935878999077495],[-77.045258,38.93592299907749],[-77.045192,38.935938999077486],[-77.045126,38.935952999077486],[-77.045092,38.935958999077485],[-77.044991,38.935972999077485],[-77.044856,38.935983999077486],[-77.044781,38.935986999077485],[-77.044268,38.93598799907749],[-77.043659,38.93598799907749],[-77.043435,38.93599199907749],[-77.04338,38.93599399907749],[-77.043324,38.93599499907749],[-77.043232,38.936000999077486],[-77.043096,38.93601299907749],[-77.043005,38.936022999077494],[-77.04287,38.93604199907748],[-77.042718,38.93607099907749],[-77.042667,38.936078999077495],[-77.042617,38.936087999077486],[-77.042362,38.93612699907749],[-77.042244,38.93613599907749],[-77.042087,38.936143999077494],[-77.041969,38.936145999077496],[-77.041726,38.93613999907749],[-77.041634,38.9361409990775],[-77.041541,38.93614699907749],[-77.041449,38.93615499907749],[-77.041404,38.93616099907749],[-77.041313,38.93617499907749],[-77.041273,38.93618299907749],[-77.041234,38.93619199907749],[-77.041117,38.936221999077496],[-77.04104,38.936245999077485],[-77.040965,38.93627099907749],[-77.040928,38.936284999077486],[-77.040843,38.93632299907749],[-77.04076,38.9363629990775],[-77.040719,38.93638499907749],[-77.040639,38.93642999907748],[-77.040563,38.936477999077496],[-77.040526,38.93650299907748],[-77.040419,38.93656699907747],[-77.040352,38.936601999077496],[-77.040245,38.93665199907748],[-77.040213,38.936664999077486],[-77.040182,38.936678999077486],[-77.040119,38.936703999077494],[-77.040022,38.93673899907748],[-77.039958,38.936759999077495],[-77.039797,38.93680699907748],[-77.039666,38.936840999077496],[-77.039525,38.93687999907748],[-77.039303,38.93693999907748],[-77.039244,38.93695799907748],[-77.039118,38.936999999077486],[-77.038994,38.937044999077486],[-77.038912,38.93707699907747],[-77.038872,38.93709399907748],[-77.038831,38.937109999077485],[-77.038621,38.93721099907748],[-77.037998,38.93756299907748],[-77.037746,38.93769899907748],[-77.037621,38.937760999077476],[-77.03743,38.93785199907748],[-77.037301,38.93790999907747],[-77.037171,38.93796599907748],[-77.037024,38.938025999077475],[-77.036859,38.93808799907748],[-77.036692,38.938147999077486],[-77.036445,38.938230999077476],[-77.036445,38.93804699907748],[-77.036446,38.93792499907749],[-77.036447,38.937729999077476],[-77.036447,38.93742699907748],[-77.035231,38.93742699907748],[-77.0348,38.93742499907747],[-77.034358,38.93742699907748],[-77.034302,38.937425999077476],[-77.034189,38.93742099907748],[-77.034133,38.93741699907748],[-77.034021,38.93740399907748],[-77.03391,38.937386999077475],[-77.033801,38.93736499907748],[-77.033721,38.93734499907749],[-77.033634,38.93732399907749],[-77.03317,38.93720499907748],[-77.03273,38.93707799907748],[-77.032259,38.93696499907749],[-77.031839,38.93684999907749],[-77.030959,38.93661899907749],[-77.030864,38.936594999077485],[-77.030025,38.9363679990775],[-77.029725,38.936284999077486],[-77.028494,38.935963999077494],[-77.028384,38.935934999077496],[-77.028048,38.935848999077486],[-77.027508,38.935700999077504],[-77.0272,38.9356189990775],[-77.027097,38.93559099907749],[-77.026726,38.935492999077496],[-77.026285,38.93537799907749],[-77.025999,38.935301999077495],[-77.025943,38.935284999077496],[-77.025893,38.9352579990775],[-77.025831,38.935209999077486],[-77.025525,38.9351309990775],[-77.025499,38.93512599907749],[-77.025447,38.9351179990775],[-77.025365,38.935107999077495],[-77.025303,38.9351059990775],[-77.025244,38.9351279990775],[-77.024431,38.93625399907748],[-77.024312,38.93642099907749],[-77.024203,38.936407999077495],[-77.02403,38.93638799907749],[-77.023353,38.93655799907748],[-77.023086,38.93662199907749],[-77.022648,38.936727999077476],[-77.02197,38.93689499907748],[-77.021469,38.937012999077496],[-77.021372,38.93702699907748],[-77.021241,38.93704599907748],[-77.019787,38.93723099907748],[-77.019752,38.937236999077484],[-77.019685,38.93724999907747],[-77.019636,38.937259999077476],[-77.019577,38.93727399907748],[-77.019518,38.93728999907748],[-77.019033,38.93739799907748],[-77.019087,38.93725599907748],[-77.019168,38.93708899907748],[-77.019193,38.93702699907748],[-77.019203,38.93699599907749],[-77.019212,38.936963999077484],[-77.019221,38.93692499907749],[-77.019226,38.93689899907748],[-77.01923,38.93680699907748],[-77.019234,38.93673399907748],[-77.01923,38.93658499907749],[-77.01922,38.9362019990775],[-77.01921,38.9358669990775],[-77.019201,38.93555799907749],[-77.019193,38.93521499907749],[-77.019187,38.9348919990775],[-77.019184,38.9348089990775],[-77.019172,38.93438499907749],[-77.019166,38.9342419990775],[-77.019164,38.9341829990775],[-77.019161,38.9341589990775],[-77.01914,38.934031999077504],[-77.019129,38.9339819990775],[-77.019111,38.9339069990775],[-77.019031,38.9336109990775],[-77.019012,38.93353399907752],[-77.018927,38.933177999077515],[-77.018804,38.93269099907751],[-77.018645,38.93205499907752],[-77.018509,38.93150099907752],[-77.018494,38.931439999077526],[-77.018418,38.93111299907753],[-77.018349,38.93082199907752],[-77.018277,38.93055599907752],[-77.018256,38.93047799907753],[-77.018178,38.930153999077525],[-77.018081,38.92973099907755],[-77.018072,38.92969599907754],[-77.018052,38.929609999077535],[-77.018,38.92939399907754],[-77.017982,38.92931399907753],[-77.017958,38.92919499907753],[-77.017951,38.92914399907754],[-77.017949,38.92910799907753],[-77.017949,38.929080999077534],[-77.017951,38.92904299907754],[-77.017953,38.92902099907754],[-77.017956,38.92899999907755],[-77.017981,38.92889099907754],[-77.018026,38.928765999077534],[-77.018042,38.92871399907754],[-77.018106,38.92848399907755],[-77.017984,38.92844899907754],[-77.017954,38.92843999907755],[-77.017887,38.928415999077544],[-77.01782,38.92838999907755],[-77.017732,38.928349999077554],[-77.017571,38.92826399907754],[-77.017549,38.92825099907755],[-77.017507,38.92822399907755],[-77.017468,38.92819599907754],[-77.017449,38.928181999077545],[-77.017394,38.92813699907755],[-77.017361,38.92810499907754],[-77.017345,38.92808799907754],[-77.017264,38.92800799907755],[-77.017165,38.92790599907756],[-77.017064,38.92780699907754],[-77.017011,38.92775699907755],[-77.016912,38.92767899907754],[-77.016749,38.92753599907755],[-77.016667,38.92748399907756],[-77.016088,38.927144999077555],[-77.015166,38.92661199907755],[-77.015074,38.92656299907757],[-77.014978,38.92651699907756],[-77.01488,38.92647599907755],[-77.014474,38.926344999077564],[-77.014422,38.92633299907756],[-77.014264,38.92630399907757],[-77.014165,38.92628899907755],[-77.014028,38.92627399907756],[-77.013889,38.92626499907756],[-77.013843,38.926263999077555],[-77.012671,38.92626699907756],[-77.012517,38.92626999907756],[-77.012466,38.926271999077564],[-77.012414,38.926272999077554],[-77.012142,38.92628199907756],[-77.012141,38.92579499907756],[-77.012144,38.92469299907756],[-77.012144,38.92419199907757],[-77.012144,38.92416199907757],[-77.012146,38.923623999077584],[-77.012146,38.92347499907758],[-77.012146,38.923376999077576],[-77.012145,38.92307099907759],[-77.012147,38.92253799907758],[-77.012146,38.92237499907758],[-77.012146,38.922000999077596],[-77.012148,38.92148099907759],[-77.012148,38.9213059990776],[-77.012257,38.92130699907759],[-77.012642,38.9213059990776],[-77.012674,38.92130399907759],[-77.012706,38.921297999077595],[-77.012736,38.9212889990776],[-77.012882,38.921215999077596],[-77.012974,38.92117199907759],[-77.013068,38.9211299990776],[-77.013212,38.92106999907759],[-77.013281,38.92104399907759],[-77.013422,38.920993999077595],[-77.013565,38.9209479990776],[-77.013637,38.9209259990776],[-77.013783,38.9208849990776],[-77.013929,38.920847999077594],[-77.014068,38.9208159990776],[-77.014278,38.92077299907759],[-77.014418,38.92074699907759],[-77.014559,38.920722999077604],[-77.01463,38.920711999077604],[-77.014935,38.9206679990776],[-77.014926,38.92061799907759],[-77.014873,38.920228999077594],[-77.014802,38.9196629990776],[-77.014749,38.91917399907761],[-77.014741,38.9190989990776],[-77.01469,38.91866299907761],[-77.014636,38.91818299907762],[-77.014625,38.9181069990776],[-77.014619,38.91806099907761],[-77.014615,38.91802299907761],[-77.014578,38.91772199907763],[-77.014575,38.91769199907761],[-77.014571,38.91763699907762],[-77.014566,38.91757999907762],[-77.014501,38.91699499907762],[-77.01442,38.916278999077626],[-77.014414,38.91622099907763],[-77.014382,38.915963999077626],[-77.014374,38.91588599907763],[-77.01434,38.91556899907763],[-77.014332,38.915482999077625],[-77.01431,38.91530999907763],[-77.014291,38.915171999077636],[-77.014278,38.915053999077635],[-77.015735,38.914556999077625],[-77.0169,38.91413699907764],[-77.017066,38.91420799907763],[-77.017326,38.914316999077634],[-77.017845,38.914530999077634],[-77.018109,38.91464599907764],[-77.018158,38.91466699907764],[-77.018414,38.914783999077635],[-77.018928,38.915000999077634],[-77.019739,38.915343999077635],[-77.019976,38.91544999907764],[-77.02004,38.915478999077635],[-77.020117,38.91550999907763],[-77.020658,38.91573799907763],[-77.020669,38.91571699907763],[-77.020682,38.91569599907763],[-77.020697,38.91567599907763],[-77.02073,38.91564299907763],[-77.020746,38.91562999907763],[-77.02077,38.915613999077635],[-77.020802,38.91559799907763],[-77.020837,38.91558499907764],[-77.020873,38.915574999077634],[-77.020872,38.91548899907764],[-77.020872,38.91526499907763],[-77.020873,38.91507999907763],[-77.020872,38.91421599907764],[-77.02087,38.91408599907765],[-77.021397,38.91408599907765],[-77.021783,38.914087999077644],[-77.021916,38.914087999077644],[-77.022945,38.91408899907764],[-77.023081,38.91408899907764],[-77.023448,38.91408599907765],[-77.023968,38.91408599907765],[-77.024306,38.914081999077645],[-77.024364,38.91409199907764],[-77.025584,38.91408699907763],[-77.025657,38.91407999907765],[-77.02599,38.91408299907764],[-77.027035,38.914089999077646],[-77.02808,38.91409299907763],[-77.02962,38.914087999077644],[-77.02975,38.914089999077646],[-77.030263,38.91408899907764],[-77.030923,38.91409199907764],[-77.031671,38.91409199907765],[-77.031782,38.91409099907764],[-77.031951,38.91409399907764],[-77.031952,38.91417999907764],[-77.031953,38.91447999907764],[-77.031952,38.914830999077644],[-77.031952,38.91526199907763],[-77.031956,38.91556799907763],[-77.031952,38.91628899907762],[-77.031951,38.91637199907763],[-77.031951,38.91699799907762],[-77.032478,38.91699599907762],[-77.033087,38.91699599907762],[-77.033246,38.916996999077625],[-77.033523,38.91699799907762],[-77.033859,38.91699899907762],[-77.033998,38.91699899907762],[-77.034039,38.91699899907762],[-77.034402,38.917002999077624],[-77.034532,38.91700199907762],[-77.035026,38.916999999077625],[-77.036177,38.91700199907762],[-77.036245,38.917002999077624],[-77.036492,38.91700199907762],[-77.036859,38.91700199907762],[-77.037477,38.91700199907762],[-77.037753,38.917002999077624],[-77.038472,38.917002999077624],[-77.03883,38.91700399907763],[-77.038941,38.91700299907762],[-77.040091,38.91700499907762],[-77.040728,38.91700499907762],[-77.041142,38.91700499907762],[-77.041224,38.917007999077605],[-77.04129,38.916968999077625],[-77.041429,38.91688999907762],[-77.041543,38.91682999907762],[-77.041656,38.91677999907762],[-77.041792,38.91671399907762],[-77.042187,38.91651999907762],[-77.042738,38.91624299907763],[-77.043345,38.91593499907763],[-77.043436,38.91588799907763],[-77.043577,38.915814999077625],[-77.04407,38.915553999077645],[-77.044257,38.91545599907762],[-77.044473,38.91534699907763],[-77.044718,38.915214999077634],[-77.044913,38.91510799907763],[-77.045039,38.91503799907763],[-77.045265,38.91490899907763],[-77.04574,38.91463999907764],[-77.045992,38.91449999907764],[-77.046088,38.914433999077644],[-77.046188,38.914760999077636],[-77.046187,38.914838999077645],[-77.046211,38.91529399907763],[-77.046228,38.915838999077636],[-77.046285,38.91599099907763],[-77.046406,38.91628899907762],[-77.046489,38.91639899907764],[-77.046598,38.916512999077625],[-77.046692,38.91667999907762],[-77.046704,38.91670299907762],[-77.04674,38.91676699907764],[-77.046774,38.91683499907762],[-77.046986,38.91724999907762],[-77.047027,38.91733099907761],[-77.047235,38.91773999907761],[-77.047522,38.91830499907761],[-77.047569,38.91839099907761],[-77.047597,38.918430999077614],[-77.047631,38.91846799907761],[-77.04765,38.9184849990776],[-77.047691,38.91851699907762],[-77.047713,38.9185319990776],[-77.047736,38.918544999077625],[-77.047789,38.918570999077616],[-77.047813,38.918579999077615],[-77.047863,38.91859499907762],[-77.04794,38.91861299907762],[-77.048176,38.9186739990776],[-77.048234,38.91869299907761],[-77.048313,38.91871699907762],[-77.048464,38.918760999077605],[-77.04852,38.918773999077615],[-77.048608,38.91879699907761],[-77.048653,38.91881099907762],[-77.048687,38.918822999077605],[-77.048732,38.91885299907761],[-77.04875,38.918866999077615],[-77.048799,38.918907999077604],[-77.048816,38.9189209990776],[-77.048872,38.91898699907762],[-77.048939,38.919109999077605],[-77.048958,38.9191409990776],[-77.049009,38.919230999077605],[-77.049025,38.91926399907761],[-77.049057,38.919323999077605],[-77.049122,38.91941899907761],[-77.04915,38.919464999077604],[-77.049188,38.9195329990776],[-77.049951,38.9208419990776],[-77.050133,38.921156999077596],[-77.050181,38.92124099907759],[-77.050055,38.921411999077584],[-77.049752,38.92169999907759],[-77.049548,38.92187499907759],[-77.04933,38.92196499907759],[-77.049313,38.92197299907759],[-77.048942,38.922203999077595],[-77.048702,38.92240499907759],[-77.04859,38.92257199907758],[-77.04854,38.922770999077585],[-77.048512,38.92304499907758],[-77.048565,38.92334399907758],[-77.048598,38.92367199907758],[-77.048625,38.92401299907757],[-77.048727,38.92448099907758],[-77.048823,38.92472499907757],[-77.048921,38.92497399907757],[-77.049225,38.925311999077564],[-77.049442,38.925564999077565],[-77.049872,38.92770299907755],[-77.049805,38.927883999077544],[-77.049669,38.92808199907754],[-77.049515,38.92813399907755],[-77.049331,38.92812999907755],[-77.049124,38.928078999077556],[-77.048944,38.92799999907755],[-77.048257,38.92740299907755],[-77.047682,38.92701199907755],[-77.047055,38.92671199907755],[-77.046732,38.92619799907756],[-77.046602,38.926055999077555],[-77.046303,38.92594299907756],[-77.046162,38.925959999077556],[-77.045534,38.92623999907756],[-77.04491,38.92657799907756],[-77.044787,38.92680399907755],[-77.044686,38.92715899907755],[-77.044699,38.92745399907755],[-77.044696,38.92748199907756],[-77.044687,38.92762299907755],[-77.044591,38.92786699907755],[-77.044561,38.92792699907754],[-77.044534,38.928122999077544],[-77.044515,38.928344999077545],[-77.044537,38.928509999077534],[-77.044683,38.928777999077546],[-77.04495,38.929050999077546],[-77.045364,38.92939599907754],[-77.045746,38.92979699907753],[-77.046373,38.93065299907753],[-77.046804,38.93110599907752],[-77.046962,38.931270999077526],[-77.047009,38.93131999907752],[-77.047137,38.93142199907752],[-77.047179,38.93144099907752],[-77.047262,38.93152199907752],[-77.047374,38.931623999077516],[-77.047798,38.93174899907751],[-77.0484,38.93190099907751],[-77.048862,38.93202899907752],[-77.049482,38.93218299907752],[-77.049742,38.93219999907752],[-77.050194,38.93222099907752],[-77.050322,38.93226899907752],[-77.050744,38.932603999077514],[-77.050854,38.932825999077515],[-77.050779,38.93300999907752]]]}}
#'

url = 'https://api.github.com/repos/unitedstates/districts/contents'
path = f'{url}/cds/2012/DC-0/shape.geojson'

from google.colab import files
uploaded = files.upload()
import json
#data = json.loads('/content/ward1.geojson')

folium.GeoJson(
    uploaded,
    name='shape.geojson'
).add_to(m_3)

# folium.Choropleth(
#     geo_data=uploaded,
#     name='choropleth',
#     #data=state_data,
#     columns=['State', 'Unemployment'],
#     key_on='feature.id',
#     fill_color='YlGn',
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='Unemployment Rate (%)'
# ).add_to(m_3)

folium.LayerControl().add_to(m_3)

m_3